In [ ]:
import os
import sys

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import plotly.express as px

# import plotly.express as px
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points  # Correct import path

import psycopg2

from datetime import datetime, timedelta

from scipy.interpolate import interp1d



In [ ]:
###Local libraries

sys.path.insert(0, '/Volumes/RyanMercerTB3/dev_RyanMercer')

from locallib.dataframe_plotly import (plot_stacked_time_series_df,
                                       plot_overlayed_time_series_df)
from locallib.dataframe_utils import (print_df_size_in_mb, 
                                     resample_data_frame)
from locallib.time_series_utils import (pad_short_time_series,
                                       nearest_neighbor_selection,
                                       plot_query_nn,
                                       plot_time_series,
                                       plot_stacked_time_series_list,)
from locallib.plotting.plot_overlayed_time_series_df import (plot_overlayed_time_series_df)
from locallib import (novelets, 
                      contrast_profile)

In [ ]:
sites_info = [
    {'enable': False, 
     'site_name': 'Calumet SWD', 
     'site_id': 33614, 'num_pumps': 1, 
     'site_dir_name': 'CalumetSWD(33614)', 
     'pump_curve_path': '/Users/audreyder/Bison_Water/AllPumpCSV/PumpCurve_Calument_33614_DataPoints.csv', # Ryan & DLT, replace these paths as appropriate - Audrey
     'calibration_stages': [
         {'frequency': 49, 
          'start_time': '2024-12-13 12:00:19', 
          'end_time': '2024-12-13 13:55:19'}, 
          
          {'frequency': 51, 
           'start_time': '2024-12-13 14:00:19', 
           'end_time': '2024-12-13 15:40:19'}, 
           
           {'frequency': 53, 
            'start_time': '2024-12-13 15:45:21', 
            'end_time': '2024-12-13 17:40:20'}, 
            
            {'frequency': 55, 
             'start_time': '2024-12-13 17:45:21', 
             'end_time': '2024-12-13 19:35:20'}]}, 
             
     {'enable': False, 
      'site_name': 'Canadian SWD', 
      'site_id': 57740, 
      'num_pumps': 1, 
      'site_dir_name': 'CanadianSWD(57740)', 
      'pump_curve_path': '/Users/audreyder/Bison_Water/AllPumpCSV/PumpCurve_Canadian_57740_DataPoints.csv', # Ryan & DLT, replace these paths as appropriate - Audrey
      'calibration_stages': [
          {'frequency': 44, 
           'start_time': '2024-12-13 12:00:02', 
           'end_time': '2024-12-13 13:58:02'}, 
           
           {'frequency': 47, 
            'start_time': '2024-12-13 13:59:03', 
            'end_time': '2024-12-13 15:51:03'}, 
            
           {'frequency': 49, 
             'start_time': '2024-12-13 15:52:03', 
             'end_time': '2024-12-13 17:51:03'}, 
             
           {'frequency': 51, 
            'start_time': '2024-12-13 17:52:02', 
            'end_time': '2024-12-13 19:50:04'}, 
            
           {'frequency': 53, 
            'start_time': '2024-12-13 19:51:03', 
            'end_time': '2024-12-13 21:47:04'}, 
            
           {'frequency': 55, 
            'start_time': '2024-12-13 21:48:03', 
            'end_time': '2024-12-13 23:53:02'}, 
            
           {'frequency': 57, 
            'start_time': '2024-12-13 23:54:03', 
            'end_time': '2024-12-14 01:53:03'}, 
            
           {'frequency': 59, 
            'start_time': '2024-12-14 01:54:03', 
            'end_time': '2024-12-14 03:53:02'}]
     }, 
            
     {'enable': False, 
      'site_name': 'Siegrist SWD', 
      'site_id': 33467, 
      'num_pumps': 1, 
      'site_dir_name': 'SiegristSWD(33467)', 
      'pump_curve_path': 'AllPumpCSV/PumpCurve_Siegrist_33467_DataPoints.csv', # Ryan & DLT, replace these paths as appropriate - Audrey
      'calibration_stages': [
            {'frequency': 47, 
             'start_time': '2024-12-17 10:00:19', 
             'end_time': '2024-12-17 12:00:21'}, 
             
            {'frequency': 49, 
             'start_time': '2024-12-17 12:05:25', 
             'end_time': '2024-12-17 14:00:32'}, 
             
            {'frequency': 51, 
             'start_time': '2024-12-17 14:01:18', 
             'end_time': '2024-12-17 16:00:19'}, 
             
            {'frequency': 53, 
             'start_time': '2024-12-17 16:01:04', 
             'end_time': '2024-12-17 18:00:38'}, 
             
            {'frequency': 55, 
             'start_time': '2024-12-17 18:00:58', 
             'end_time': '2024-12-17 20:00:22'}, 
             
            {'frequency': 57, 
             'start_time': '2024-12-17 20:05:27', 
             'end_time': '2024-12-17 21:55:19'}]
             
     }, 
     
     {'enable': False, 
      'site_name': 'Union City 2 SWD', 
      'site_id': 33404, 
      'num_pumps': 1, 
      'site_dir_name': 'UnionCity2SWD(33404)', 
      'pump_curve_path': '/Users/audreyder/Bison_Water/AllPumpCSV/PumpCurve_UnionCity2_33404_DataPoints.csv', # Ryan & DLT, replace these paths as appropriate - Audrey
      'calibration_stages': [
          {'frequency': 46, 
           'start_time': '2024-12-17 12:00:12', 
           'end_time': '2024-12-17 13:50:12'}, 
           
          {'frequency': 48, 
           'start_time': '2024-12-17 13:55:11', 
           'end_time': '2024-12-17 15:50:11'}, 
           
          {'frequency': 50, 
           'start_time': '2024-12-17 15:55:11', 
           'end_time': '2024-12-17 17:50:12'}, 
           
          {'frequency': 52, 
           'start_time': '2024-12-17 17:55:12', 
           'end_time': '2024-12-17 19:50:11'}, 
           
          {'frequency': 54, 
           'start_time': '2024-12-17 19:55:11', 
           'end_time': '2024-12-17 21:45:11'}, 
           
          {'frequency': 56, 
           'start_time': '2024-12-17 21:50:11', 
           'end_time': '2024-12-17 23:35:14'}]
           
     }
     
]

In [ ]:
#     '57740': 'Canadian SWD',
#     '33614': 'Calumet SWD',
#     '33404': 'Union City 2 SWD',
#     '33467': 'Siegrist SWD',
#     # '50248': 'Seiling SWD',
#     # '57651': 'American Horse SWD',
#     # '60818': 'Brown SWD',
#     # '60385': 'BIG 4 SWD',
#     # '53764': 'Okarche South SWD',
#     # '54563': 'Okarche North SWD',
#     # '50208': 'Alpha SWD',
#     # '50252': 'Red Winter SWD ',
#     # '54727': 'Kingfisher SWD Pump 1',
#     # '54728': 'Kingfisher SWD Pump 2',
#     # '48054': '3 mile SWD',

# sites_info = [
#     {'site_name':'Canadian SWD',   'site_id': 57740, 'num_pumps': 1, 'site_dir_name': 'CanadianSWD(57740)'},
#     # {'site_name':'Union City 2 SWD', 'site_id': 33404, 'num_pumps': 1, 'site_dir_name': 'UnionCity2SWD(33404)'},
#     # {'site_name':'Siegrist SWD',   'site_id': 33467, 'num_pumps': 1, 'site_dir_name': 'SiegristSWD(33467)'},
#     # {'site_name':'Calumet SWD',   'site_id': 33614, 'num_pumps': 1, 'site_dir_name': 'CalumetSWD(33614)'},
# ]

In [ ]:
# Replace 'path_to_csv_file.csv' with the path to your CSV file
enable_site_idx = 3
for site_idx in range(len(sites_info)):
    sites_info[site_idx]['enable'] = site_idx == enable_site_idx
    
site_info = sites_info[enable_site_idx]
csv_file_path = site_info['pump_curve_path']

# Load the data from the CSV file
df_pump_data = pd.read_csv(csv_file_path)

In [ ]:
# device_facility_map = {
    
#     '57740': 'Canadian SWD',
#     '33614': 'Calumet SWD',
#     '33404': 'Union City 2 SWD',
#     '33467': 'Siegrist SWD',
#     # '50248': 'Seiling SWD',
#     # '57651': 'American Horse SWD',
#     # '60818': 'Brown SWD',
#     # '60385': 'BIG 4 SWD',
#     # '53764': 'Okarche South SWD',
#     # '54563': 'Okarche North SWD',
#     # '50208': 'Alpha SWD',
#     # '50252': 'Red Winter SWD ',
#     # '54727': 'Kingfisher SWD Pump 1',
#     # '54728': 'Kingfisher SWD Pump 2',
#     # '48054': '3 mile SWD',
    
    
#     # Add more device IDs and facility names here
# }

# 1. Device Selection Options
# [
#     57740, #'Canadian SWD'
#     # 48034,
#     # 33614, #'Calumet SWD',
#     # 33404, #'Union City 2 SWD'
#     # 33467, #'Siegrist SWD'    
# ]  # Li
device_ids = [site_info['site_id'] for site_info in sites_info if site_info['enable']==True]

# st of device IDs (integers), e.g., [57740, 33614]
device_name_substrings = []#['canadian']  # List of devicename substrings, e.g., ['canadian', 'alpha']

# 2. Time Zone
time_zone = 'UTC'

# 3. Measurement Mappings
measurements = {
    # 'Vibration': ['sv/pmp_vib_rear', 'sv/hp1_vib', 'sv/vib'],
    # 'Thrust Temperature': ['sv/thrust_temp'],
    # 'Suction Pressure': ['sv/suctp'],
    'Discharge Pressure': ['sv/discp'],
    'Flow Rate': ['sv/fr'],
    'Frequency': ['sv/vfd_speed', 'sv/HZ', 'sv/hp1_hz'],
    # 'Amps': ['sv/vfd_current'],
    # 'AmpA': ['sv/hp1_amps_a'],
    # 'AmpsB': ['sv/hp1_amps_b'],
    # 'AmpsC': ['sv/hp1_amps_c'],
    # 'Volts': ['sv/vfd_voltage', 'sv/hp1_volts_a', 'sv/hp1_volts_b', 'sv/hp1_volts_c'],
    # 'VoltsA': ['sv/hp1_volts_a'],
    # 'VoltsB': ['sv/hp1_volts_b'],
    # 'VoltsC': ['sv/hp1_volts_c'],
    # 'Meter Total': ['sv/accum_volume', 'sv/av'],
    # Add more measurements as needed
}

# 4. Time Range
# time_interval = '1 hour'  # Adjust as needed

# Alternatively, specify exact start and end times
time_interval = None
# start_time = datetime.now() - timedelta(days=10)
# end_time = datetime.now()

start_time = '2024-12-10'
end_time = '2025-01-10'

# Function to generate SQL query
def generate_sql_query(device_ids, device_name_substrings, time_zone, measurements, time_interval):
    # Build SELECT clauses for each measurement
    select_clauses = [
        "h.deviceid AS site_id",
        "d.devicename AS facility_name",
        "CONVERT_TIMEZONE(%s, h.datetime) AS timestamp"
    ]
    params = [time_zone]
    where_paths = set()

    for measurement_name, paths in measurements.items():
        case_conditions = "\n            ".join(
            [f"WHEN h.path = '{path}' THEN h.value" for path in paths]
        )
        select_clause = f"""MAX(
                CASE
                    {case_conditions}
                    ELSE NULL
                END
            ) AS "{measurement_name}" """
        select_clauses.append(select_clause)
        where_paths.update(paths)

    # Build WHERE clause components
    where_clauses = []
    where_params = []

    # Device ID filtering
    if device_ids:
        device_id_placeholders = ', '.join(['%s'] * len(device_ids))
        where_clauses.append(f"h.deviceid IN ({device_id_placeholders})")
        where_params.extend(device_ids)

    # Device name substring filtering
    if device_name_substrings:
        name_conditions = []
        for substring in device_name_substrings:
            name_conditions.append("d.devicename ILIKE %s")
            where_params.append(f"%{substring}%")
        where_clauses.append("(" + " OR ".join(name_conditions) + ")")

    # Only include devices that are not disabled
    where_clauses.append("d.disabled = %s")
    where_params.append(False)

    # h.path filtering
    path_placeholders = ', '.join(['%s'] * len(where_paths))
    where_clauses.append(f"h.path IN ({path_placeholders})")
    where_params.extend(where_paths)

    # Time filtering
    # where_clauses.append(f"h.datetime > GETDATE() - INTERVAL %s")
    # where_params.append(time_interval)
    # Alternatively, for specific times:
    where_clauses.append(f"h.datetime BETWEEN %s AND %s")
    where_params.append(start_time)
    where_params.append(end_time)

    # Combine WHERE clause
    where_clause = "WHERE\n    " + "\n    AND ".join(where_clauses)

    # Assemble the final query
    query = f"""
SELECT
    {',    '.join(select_clauses)}
FROM
    historical AS h
    JOIN devices AS d ON h.deviceid = d.deviceid
{where_clause}
GROUP BY
    h.deviceid, d.devicename, h.datetime
ORDER BY
    h.deviceid, h.datetime;
"""
    # Combine all parameters
    params.extend(where_params)
    return query, params

# Generate the query and parameters
query, params = generate_sql_query(device_ids, device_name_substrings, time_zone, measurements, time_interval)
# print("Generated SQL Query:")
# print(query)
# print("Query Parameters:")
# print(params)

# Set up your database connection (replace with your actual credentials)
conn = psycopg2.connect(
    host=os.getenv('REDSHIFT_ENDPOINT'),
    port=os.getenv('REDSHIFT_PORT'),
    database=os.getenv('REDSHIFT_DBNAME'),
    user=os.environ.get('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASS')
)

# Execute the query and load results into a pandas DataFrame
df = pd.read_sql_query(query, conn, params=params)
df['pump_id'] = 1

# Close the database connection
conn.close()

df.set_index(['site_id','pump_id','timestamp'],inplace=True)

# Display the DataFrame
# print(df)

## Data Filtering

In [ ]:
# If you need to ensure 'timestamp' is a datetime, reassign the index:
df.index = pd.MultiIndex.from_arrays(
    [
        df.index.get_level_values('site_id'),
        df.index.get_level_values('pump_id'),
        pd.to_datetime(df.index.get_level_values('timestamp'))
    ],
    names=['site_id', 'pump_id', 'timestamp']
)

# If you need a date filter, you can do something like:
# start_date = '2024-10-10'
# end_date = df.index.get_level_values('timestamp').max()
# df = df.loc[(slice(None), slice(None), slice(start_date, end_date)), :]

# Define pump-related variables
pressure_column = 'discharge pressure'
flow_column = 'flow rate'
frequency_float_column = 'frequency'
frequency_int_column = 'frequency int'

freq_min = 40
freq_max = 60

# Apply filtering masks on the main DataFrame (df)
mask = (df[frequency_float_column].astype(float) >= freq_min) & (df[frequency_float_column].astype(float) <= freq_max)
mask &= df[flow_column].astype(float) < 30000
mask &= df[flow_column].astype(float) > 5000
# If needed:
# mask &= df[pressure_column].astype(float) > 1300

df = df[mask]

df[frequency_int_column] = df[frequency_float_column].round().astype('Int64')

In [ ]:
# Define the necessary column names
flow_column = 'flow rate'          # Replace with your actual flow rate column name
pressure_column = 'discharge pressure'  # Replace with your actual pressure (TDH) column name
frequency_column = 'frequency'     # Replace with your actual frequency column name
frequency_int_column = 'frequency_int'  # If you have integer frequencies
timestamp_column = 'timestamp'     # Replace with your actual timestamp column name

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Separate speed and efficiency lines
speed_data = df_pump_data[df_pump_data['type'] == 'speed']
efficiency_data = df_pump_data[df_pump_data['type'] == 'efficiency']

# Get unique labels for speed and efficiency lines
speed_lines = sorted(speed_data['label'].unique(), key=lambda x: float(x))
efficiency_lines = [line for line in sorted(efficiency_data['label'].unique()) if 'triangle' not in line.lower()]

# Create interpolation functions for each speed line
speed_line_funcs = {}
for label in speed_lines:
    line_data = speed_data[speed_data['label'] == label].sort_values('x')
    x_speed = line_data['x'].values
    y_speed = line_data['y'].values
    speed_line_funcs[label] = interp1d(x_speed, y_speed, bounds_error=False, fill_value='extrapolate')

# Create interpolation functions for each efficiency line
efficiency_line_funcs = {}
for label in efficiency_lines:
    line_data = efficiency_data[efficiency_data['label'] == label].sort_values('x')
    x_efficiency = line_data['x'].values
    y_efficiency = line_data['y'].values
    efficiency_line_funcs[label] = interp1d(x_efficiency, y_efficiency, bounds_error=False, fill_value='extrapolate')

# Find intersection points between speed lines and efficiency lines
intersection_points = {}

def find_intersection_point(func1, func2, x_min, x_max):
    from scipy.optimize import brentq
    def func_diff(x):
        return func1(x) - func2(x)
    try:
        x_intersect = brentq(func_diff, x_min, x_max)
        y_intersect = func1(x_intersect)
        return x_intersect, y_intersect
    except ValueError:
        return None

for speed_line in speed_lines:
    speed_func = speed_line_funcs[speed_line]
    x_speed_min = speed_data[speed_data['label'] == speed_line]['x'].min()
    x_speed_max = speed_data[speed_data['label'] == speed_line]['x'].max()
    intersection_points[speed_line] = {}
    for efficiency_line in efficiency_lines:
        eff_func = efficiency_line_funcs[efficiency_line]
        x_eff_min = efficiency_data[efficiency_data['label'] == efficiency_line]['x'].min()
        x_eff_max = efficiency_data[efficiency_data['label'] == efficiency_line]['x'].max()
        x_min = max(x_speed_min, x_eff_min)
        x_max = min(x_speed_max, x_eff_max)
        if x_min < x_max:
            intersection = find_intersection_point(speed_func, eff_func, x_min, x_max)
            if intersection is not None:
                intersection_points[speed_line][efficiency_line] = intersection
            else:
                print(f"No intersection found between speed line {speed_line} and efficiency line {efficiency_line}")
        else:
            print(f"No overlapping x range between speed line {speed_line} and efficiency line {efficiency_line}")

# Interpolate between Min to BEP and BEP to Max for each speed line
interpolated_speed_line = {}

for speed_line in speed_lines:
    if all(key in intersection_points[speed_line] for key in ['Min', 'BEP', 'Max']):
        min_point = intersection_points[speed_line]['Min']
        bep_point = intersection_points[speed_line]['BEP']
        max_point = intersection_points[speed_line]['Max']
    else:
        print(f"Missing intersection points for speed line {speed_line}, skipping")
        continue

    speed_func = speed_line_funcs[speed_line]

    # Interpolate between Min and BEP
    x_min_bep = np.linspace(min_point[0], bep_point[0], 100)
    y_min_bep = speed_func(x_min_bep)
    health_score_min_bep = np.linspace(-100, 0, 100)

    # Interpolate between BEP and Max
    x_bep_max = np.linspace(bep_point[0], max_point[0], 100)
    y_bep_max = speed_func(x_bep_max)
    health_score_bep_max = np.linspace(0, 100, 100)

    # Combine the two
    x_interp = np.concatenate([x_min_bep, x_bep_max])
    y_interp = np.concatenate([y_min_bep, y_bep_max])
    health_score = np.concatenate([health_score_min_bep, health_score_bep_max])

    interpolated_speed_line[speed_line] = pd.DataFrame({
        'flow_rate': x_interp,
        'tdh': y_interp,
        'health_score': health_score
    })

# Create interpolation functions for each speed line
health_score_functions = {}
for speed_line in interpolated_speed_line:
    df_line = interpolated_speed_line[speed_line]
    df_line = df_line.drop_duplicates(subset='flow_rate').sort_values('flow_rate')
    flow_rate = df_line['flow_rate'].values
    health_score = df_line['health_score'].values
    health_score_func = interp1d(
        flow_rate, health_score, bounds_error=False, fill_value=(health_score[0], health_score[-1])
    )
    health_score_functions[speed_line] = health_score_func

# Extract numeric frequencies from speed lines
speed_line_freqs = np.array([float(speed_line) for speed_line in speed_lines])

# Function to compute health score for each time sample
def compute_health_score(row):
    f = row[frequency_column]
    q = row[flow_column]

    # Check if frequency is within the desired range
    if f < freq_min or f > freq_max:
        return np.nan  # Exclude frequencies outside the range

    # Find two nearest speed lines
    freq_array = speed_line_freqs
    if f <= freq_array.min():
        f1 = f2 = freq_array.min()
    elif f >= freq_array.max():
        f1 = f2 = freq_array.max()
    else:
        idx = np.searchsorted(freq_array, f)
        f1 = freq_array[idx - 1]
        f2 = freq_array[idx]

    f1_str = str(int(f1)) if f1 == int(f1) else str(f1)
    f2_str = str(int(f2)) if f2 == int(f2) else str(f2)

    # Compute weights
    if f1 == f2:
        weight1 = 1.0
        weight2 = 0.0
    else:
        weight1 = (f2 - f) / (f2 - f1)
        weight2 = (f - f1) / (f2 - f1)

    # Get health scores at flow rate q
    try:
        health_score_f1 = health_score_functions[f1_str](q)
    except KeyError:
        print(f"Speed line {f1_str} not found")
        return np.nan
    except ValueError:
        return np.nan  # Flow rate q is outside interpolation range
    try:
        health_score_f2 = health_score_functions[f2_str](q)
    except KeyError:
        print(f"Speed line {f2_str} not found")
        return np.nan
    except ValueError:
        return np.nan  # Flow rate q is outside interpolation range

    # Compute final health score
    health_score = weight1 * health_score_f1 + weight2 * health_score_f2
    return health_score

# Assume 'df' is already indexed by ['site_id','pump_id','timestamp'] and has columns:
# flow_column, pressure_column, frequency_column, and timestamp_column

# Convert frequency to integer if necessary
df[frequency_int_column] = df[frequency_column].astype(int)

# Filter the DataFrame to only include frequencies between the lowest and highest speed lines
freq_min = speed_line_freqs.min()
freq_max = speed_line_freqs.max()
df = df[(df[frequency_column] >= freq_min) & (df[frequency_column] <= freq_max)]

# Update the list of frequencies after filtering
df[frequency_int_column] = df[frequency_column].astype(int)
frequencies = sorted(df[frequency_int_column].unique())

# Apply the function to compute health score for each row
df['health_score'] = df.apply(compute_health_score, axis=1)

# Remove rows with NaN health scores (which may result from frequencies outside the range)
df = df.dropna(subset=['health_score'])

# Create df_plot with timestamp as the index while minimally changing the code
df_plot = df.reset_index().set_index([timestamp_column])
df_plot[frequency_int_column] = df_plot[frequency_column].astype(int)

## Plots

### Colormapping

In [ ]:
# Create color map for frequencies over 40 to 60 Hz
# Frequencies outside 40-60 Hz are already excluded


frequencies = np.arange(60,39,-1)# sorted([f for f in df_plot[frequency_int_column].unique() if freq_min <= f <= freq_max],reverse=True)
freq_min = np.min(frequencies)
freq_max = np.max(frequencies)


# Define the colorscale
colorscale = px.colors.sequential.Turbo
n_colors = len(colorscale)

# Function to map frequency to color
def map_frequency_to_color(freq):
    # Map to color index
    color_index = int((freq - freq_min) / (freq_max - freq_min) * (n_colors - 1))
    return colorscale[color_index]

# Create color_map, mapping frequencies to colors
color_map = {freq: map_frequency_to_color(freq) for freq in frequencies}
# color_map = {freq: '#000000' for freq in frequencies}

### Pump Curve

In [ ]:
##################
### Pump Curve ###
##################
import plotly.graph_objects as go

plot_slice = 100

# Iterate over each site_id and pump_id pair
for (site_id, pump_id), group_df in df.groupby(level=['site_id','pump_id']):
    fig1 = go.Figure()

    # Plot speed lines
    for label in sorted(speed_lines, reverse=True, key=lambda x: float(x)):
        line_data = speed_data[speed_data['label'] == label]
        x_values = line_data['x']
        y_values = line_data['y']
        freq = int(label)
        if freq not in color_map:
            continue
        line_color = color_map[freq]
        fig1.add_trace(
            go.Scatter(
                x=x_values,
                y=y_values,
                mode='lines',
                name=f'{label}Hz',
                legendgroup=label,
                showlegend=True,
                line=dict(dash='dash', color=line_color)
            )
        )

    # Plot efficiency lines
    for label in ['Min', 'BEP', 'Max']:
        if label in efficiency_lines:
            line_data = efficiency_data[efficiency_data['label'] == label]
            x_values = line_data['x']
            y_values = line_data['y']
            fig1.add_trace(
                go.Scatter(
                    x=x_values,
                    y=y_values,
                    mode='lines',
                    name=label,
                    legendgroup=label,
                    showlegend=False,
                    line=dict(dash='solid', color='gray')
                )
            )

    # Plot synthetic data points (using the subset group_df)
    current_frequencies = sorted(group_df[frequency_int_column].unique())
    for freq in current_frequencies:
        freq_data = group_df[group_df[frequency_int_column] == freq]
        if freq not in color_map:
            continue
        fig1.add_trace(
            go.Scatter(
                x=freq_data[flow_column][::plot_slice],
                y=freq_data[pressure_column][::plot_slice],
                mode='markers',
                name=f'{int(freq)}Hz Data',
                legendgroup=f'{int(freq)}',
                marker=dict(color=color_map[freq]),
                showlegend=True
            )
        )

    # Update layout for the current site_id and pump_id
    fig1.update_layout(
        title=f'Pump Curve over Time (Site: {site_id}, Pump: {pump_id})',
        xaxis_title='Flow Rate (BBL/day)',
        yaxis_title='TDH (PSI)',
        template='plotly_white',
        height=800,
        width=950,
    )

    # Display the figure for this site_id, pump_id pair
    fig1.show()

### Time series of Percent Deviation from BEP

Shown as a scatterplot with markers color coded to the frequency 

In [ ]:
#################
### Health Score Time Series ###
#################
plot_slice = 10

# start_time = '2024-12-13'
# end_time = '2024-12-15'
# Iterate over each site_id and pump_id pair
for (site_id, pump_id), group_df in df.groupby(level=['site_id','pump_id']):
    # For each group, reset index to timestamp
    df_plot = group_df.reset_index().set_index(timestamp_column)#.loc[start_time:end_time]

    fig3 = go.Figure()

    # Get the unique frequencies for the current group
    current_frequencies = sorted(df_plot[frequency_int_column].unique(), reverse=True)

    for freq in current_frequencies:
        freq_data = df_plot[df_plot[frequency_int_column] == freq]
        # Only plot if freq is in color_map
        if freq not in color_map:
            continue
        fig3.add_trace(
            go.Scatter(
                x=freq_data[::plot_slice].index,  # Use index as x-axis (timestamp)
                y=freq_data['health_score'][::plot_slice],
                mode='markers',
                name=f'{int(freq)}Hz',
                marker=dict(color=color_map[freq]),
                legendgroup=f'{int(freq)}',
            )
        )

    # Update layout for the current site_id and pump_id
    fig3.update_layout(
        title=f'Health Score over Time (Site: {site_id}, Pump: {pump_id})',
        xaxis_title='Timestamp',
        yaxis_title='Health Score',
        template='plotly_white',
        height=600,
        width=1600,
    )

    # Display the figure for this site_id, pump_id pair
    fig3.show()


In [ ]:
# Columns in your dataframe
timestamp_column = 'timestamp'
# frequency_int_column = 'frequency'
health_score_column = 'health_score'
normalized_health_score_column = 'normalized_health_score'

calibration_stages = site_info['calibration_stages']

plot_slice = 1


def interpolate_missing_freqs(freq_dict):
    """
    Interpolates means and stds for missing frequencies.
    freq_dict should be {freq: {'mean':..., 'std':...}}
    """
    freqs = sorted(freq_dict.keys())
    if len(freqs) <= 1:
        # If we have only one or zero frequencies, no interpolation is possible
        return freq_dict

    full_freq_range = range(freqs[0], freqs[-1] + 1)

    for f in full_freq_range:
        if f not in freq_dict:
            # Find lower and higher available frequencies
            lower_freqs = [ff for ff in freqs if ff < f]
            higher_freqs = [ff for ff in freqs if ff > f]
            if not lower_freqs or not higher_freqs:
                # Cannot interpolate if no lower or higher freq exists
                continue
            lower_f = max(lower_freqs)
            higher_f = min(higher_freqs)
            # Linear interpolation of mean and std
            w = (f - lower_f) / (higher_f - lower_f)
            mean_val = freq_dict[lower_f]['mean'] + w * (freq_dict[higher_f]['mean'] - freq_dict[lower_f]['mean'])
            std_val = freq_dict[lower_f]['std'] + w * (freq_dict[higher_f]['std'] - freq_dict[lower_f]['std'])
            freq_dict[f] = {
                'mean': mean_val,
                'std': std_val
            }

    return freq_dict

# Iterate over each site_id and pump_id pair
for (site_id, pump_id), group_df in df.groupby(level=['site_id','pump_id']):
    # Reset index to timestamp
    df_plot = group_df.reset_index().set_index(timestamp_column)
    df_plot.index = pd.to_datetime(df_plot.index)

    # Compute mean and std from calibration intervals
    freq_dict = {}
    for stage in calibration_stages:
        freq = stage['frequency']
        start = pd.to_datetime(stage['start_time'])
        end = pd.to_datetime(stage['end_time'])
        
        # Filter the data for this frequency and time window
        freq_cal_data = df_plot[
            (df_plot.index >= start) &
            (df_plot.index <= end)
        ]
        
        freq_mean = freq_cal_data[health_score_column].mean() if not freq_cal_data.empty else np.nan
        freq_std = freq_cal_data[health_score_column].std() if not freq_cal_data.empty else np.nan
        
        freq_dict[freq] = {
            'mean': freq_mean,
            'std': freq_std
        }

    # Interpolate missing frequencies if needed
    freq_dict = interpolate_missing_freqs(freq_dict)

    # Normalize the health score
    def normalize_health_score(row):
        f = row[frequency_int_column]
        if f not in freq_dict:
            return np.nan
        mean_val = freq_dict[f]['mean']
        std_val = freq_dict[f]['std']
        if pd.isna(mean_val) or pd.isna(std_val) or std_val == 0:
            return np.nan
        return (row[health_score_column] - mean_val) / std_val

    df_plot[normalized_health_score_column] = df_plot.apply(normalize_health_score, axis=1)

    # At this point, df_plot has 'normalized_health_score' and is indexed by timestamp.
    # We now reset the index so we can merge back into df.
    df_plot_reset = df_plot.reset_index()
    # Now df_plot_reset has columns ['timestamp', 'site_id', 'pump_id', 'health_score', 'normalized_health_score', ...]
    # Set the multi-index to match original df index
    df_plot_reset.set_index(['site_id', 'pump_id', timestamp_column], inplace=True)

    # Merge normalized values back into df
    df.loc[df_plot_reset.index, normalized_health_score_column] = df_plot_reset[normalized_health_score_column]

    # Now that df is updated, we can proceed to plotting from df or df_plot
    current_frequencies = sorted(df_plot[frequency_int_column].unique(), reverse=True)

    fig3 = go.Figure()

    # start_time = '2024-12-13'
    # end_time = '2024-12-15'
    for freq in current_frequencies:
        freq_data = df_plot[df_plot[frequency_int_column] == freq]#.loc[start_time:end_time]

        # Use the previously defined color_map
        if freq not in color_map:
            continue

        fig3.add_trace(
            go.Scatter(
                x=freq_data[::plot_slice].index,
                y=freq_data[normalized_health_score_column][::plot_slice],
                mode='markers',
                name=f'{int(freq)}Hz',
                marker=dict(color=color_map[freq]),
                legendgroup=f'{int(freq)}',
            )
        )

    fig3.update_layout(
        title=f'Normalized Health Score over Time (Site: {site_id}, Pump: {pump_id})',
        xaxis_title='Timestamp',
        yaxis_title='Normalized Health Score',
        template='plotly_white',
        height=600,
        width=1600,
    )
    fig3.update_xaxes(showgrid=True, gridcolor='lightgray')
    fig3.update_yaxes(showgrid=True, gridcolor='lightgray')

    fig3.show()

## Moving Mean +- 3STD

In [ ]:
# normalized_health_score_column = 'health_score'
normalized_health_score_column = 'normalized_health_score'

In [ ]:
df.reset_index(inplace=True)
df.set_index(['site_id','pump_id','timestamp'],inplace=True)

In [ ]:
# Define parameters near the top
alert_threshold = 2


# Define rolling window parameters
long_window = '14D'
short_window = '6H'

# Iterate over each site_id and pump_id pair
results = {}  # Store computed data, including events, for each site/pump

for (site_id, pump_id), group_df in df.groupby(level=['site_id', 'pump_id']):
    # Reset index to timestamp
    df_plot = group_df.reset_index().set_index(timestamp_column).sort_index()

    # Compute long rolling statistics (for thresholds)
    rolling_long_mean = df_plot[normalized_health_score_column].rolling(long_window, closed='left').mean()
    rolling_long_std = df_plot[normalized_health_score_column].rolling(long_window, closed='left').std()
    upper_bound = rolling_long_mean + alert_threshold * rolling_long_std
    lower_bound = rolling_long_mean - alert_threshold * rolling_long_std

    # Compute short rolling statistic (used for event detection)
    rolling_short_mean = df_plot[normalized_health_score_column].rolling(short_window, closed='left').mean()

    # Detect events
    # Event starts when rolling_short_mean goes outside the threshold (above upper or below lower)
    # Event ends when rolling_short_mean returns inside the threshold.
    out_of_range = (rolling_short_mean > upper_bound) | (rolling_short_mean < lower_bound)
    df.reset_index(inplace=True)
    df.set_index(['timestamp'],inplace=True)
    df['anomaly'] = out_of_range
    df.reset_index(inplace=True)
    df.set_index(['site_id','pump_id','timestamp'],inplace=True)
    event_intervals = []
    in_event = False
    event_start = None
    
    for t, outside in out_of_range.items():
        if outside and not in_event:
            # Start event
            in_event = True
            event_start = t
        elif not outside and in_event:
            # End event
            in_event = False
            event_end = t
            event_intervals.append((event_start, event_end))
            
    # If still in_event by the end, close it at the last timestamp
    if in_event:
        event_end = df_plot.index[-1]
        event_intervals.append((event_start, event_end))

    # Store computed data and events
    results[(site_id, pump_id)] = {
        'df_plot': df_plot,
        'rolling_long_mean': rolling_long_mean,
        'rolling_long_std': rolling_long_std,
        'upper_bound': upper_bound,
        'lower_bound': lower_bound,
        'rolling_short_mean': rolling_short_mean,
        'event_intervals': event_intervals
    }

In [ ]:
# Second Code Cell (Plotting)
plot_slice = 10

for (site_id, pump_id), res in results.items():
    df_plot = res['df_plot']
    rolling_long_mean = res['rolling_long_mean']
    rolling_long_std = res['rolling_long_std']
    upper_bound = res['upper_bound']
    lower_bound = res['lower_bound']
    rolling_short_mean = res['rolling_short_mean']
    event_intervals = res['event_intervals']

    fig3 = go.Figure()

    # Get unique frequencies for the current group
    current_frequencies = sorted(df_plot[frequency_int_column].unique(), reverse=True)

    for freq in current_frequencies:
        freq_data = df_plot[df_plot[frequency_int_column] == freq]

        # Only plot if freq is in color_map
        if freq not in color_map:
            continue

        # Add scatter plot for raw health scores
        fig3.add_trace(
            go.Scatter(
                x=freq_data[::plot_slice].index,  # Use index as x-axis (timestamp)
                y=freq_data[normalized_health_score_column][::plot_slice],
                mode='markers',
                name=f'{int(freq)}Hz',
                marker=dict(color=color_map[freq]),
                legendgroup=f'{int(freq)}',
            )
        )

    # Plot the long rolling mean and thresholds once (they are frequency-agnostic)
    fig3.add_trace(
        go.Scatter(
            x=rolling_long_mean.index,
            y=rolling_long_mean,
            mode='lines',
            name='Long Mean',
            line=dict(color='#FFFFFF', width=5),
            legendgroup='stats',
            showlegend=True
        )
    )
    fig3.add_trace(
        go.Scatter(
            x=rolling_long_mean.index,
            y=rolling_long_mean,
            mode='lines',
            name='Long Mean',
            line=dict(color='#FF0000', width=1, dash='dash'),
            legendgroup='stats',
            showlegend=True
        )
    )

    fig3.add_trace(
        go.Scatter(
            x=upper_bound.index,
            y=upper_bound,
            mode='lines',
            name='± Threshold',
            line=dict(color='#FF0000', width=1, dash='dash'),
            legendgroup='stats',
            showlegend=True
        )
    )
    fig3.add_trace(
        go.Scatter(
            x=lower_bound.index,
            y=lower_bound,
            mode='lines',
            name='± Threshold',
            line=dict(color='#FF0000', width=1, dash='dash'),
            legendgroup='stats',
            showlegend=False
        )
    )

    # Add short rolling mean
    fig3.add_trace(
        go.Scatter(
            x=rolling_short_mean.index,
            y=rolling_short_mean,
            mode='lines',
            name='Short Mean',
            line=dict(color='black', width=2),
            legendgroup='stats',
            showlegend=True
        )
    )

    # Add event annotations
    # We'll represent each event interval as a filled area. To make them toggle together,
    # use a single legendgroup and only showlegend for the first event.
    show_legend_for_events = True
    y_min = min(df_plot[normalized_health_score_column].min(), lower_bound.min(), upper_bound.min())
    y_max = max(df_plot[normalized_health_score_column].max(), upper_bound.max(), lower_bound.max())

    for (start, end) in event_intervals:
        fig3.add_trace(
            go.Scatter(
                x=[start, start, end, end],
                y=[y_min, y_max, y_max, y_min],
                fill='toself',
                fillcolor='rgba(255,0,0,0.1)',  # red with 10% alpha
                line=dict(width=0),
                name='Events' if show_legend_for_events else None,
                legendgroup='Events',
                showlegend=show_legend_for_events
            )
        )
        show_legend_for_events = False

    # Update layout for the current site_id and pump_id
    fig3.update_layout(
        title=f'Health Score over Time (Site: {site_id}, Pump: {pump_id})',
        xaxis_title='Timestamp',
        yaxis_title='Health Score',
        template='plotly_white',
        height=500,
        width=1600,
    )

    fig3.show()

In [ ]:
# Initialize a list to collect event details
event_details = []

for (site_id, pump_id), res in results.items():
    df_plot = res['df_plot']
    rolling_short_mean = res['rolling_short_mean']
    upper_bound = res['upper_bound']
    lower_bound = res['lower_bound']
    event_intervals = res['event_intervals']
    
    for (start, end) in event_intervals:
        # Determine the event type based on the rolling_short_mean at the start of the event
        try:
            value_at_start = rolling_short_mean.loc[start]
            upper = upper_bound.loc[start]
            lower = lower_bound.loc[start]
        except KeyError:
            # Handle cases where the start time might not align perfectly
            value_at_start = rolling_short_mean.iloc[rolling_short_mean.index.get_loc(start, method='nearest')]
            upper = upper_bound.iloc[rolling_short_mean.index.get_loc(start, method='nearest')]
            lower = lower_bound.iloc[rolling_short_mean.index.get_loc(start, method='nearest')]
        
        if value_at_start > upper:
            event_type = 'Above Threshold'
        elif value_at_start < lower:
            event_type = 'Below Threshold'
        else:
            event_type = 'Unknown'  # Fallback in case it's neither
        
        # Calculate duration
        duration = end - start
        
        # Append the event details to the list
        event_details.append({
            'Site ID': site_id,
            'Pump ID': pump_id,
            'Start Time': start,
            'End Time': end,
            'Duration': duration,
            'Event Type': event_type
        })

# Convert the list of events to a DataFrame
events_df = pd.DataFrame(event_details)

# Optional: Format the duration as total hours or another preferred unit
events_df['Duration (Hours)'] = events_df['Duration'].dt.total_seconds() / 3600

# Rearrange columns for better readability
events_df = events_df[[
    'Site ID',
    'Pump ID',
    'Start Time',
    'End Time',
    'Duration (Hours)',
    'Event Type'
]]

# Display the events DataFrame
events_df

In [ ]:
# Define the output base path
output_base_path = '/Volumes/RyanMercerTB3/dev_RyanMercer/datasets/Bison/DailyReports/PercentFromBEP/'  # Replace with your desired output path

# Ensure the output base path exists
os.makedirs(output_base_path, exist_ok=True)

# Get the current date
date = datetime.now().strftime('%Y-%m-%d')

target_column = 'health_score'
annotation_column = 'anomaly'



# **Output Results to CSV Files**
for site_info in sites_info:
    site_id = site_info['site_id']
    site_name = site_info['site_name']

    # Get the list of pump_ids for the current site_id
    pump_ids = df.xs(site_id, level='site_id').index.get_level_values('pump_id').unique()

    # Sanitize the site_name for the file name (if necessary)
    site_name_clean = str(site_name).replace('/', '_').replace('(', '').replace(')', '').replace(' ', '_')

    for pump_id in pump_ids:
        # Select data for the current site and pump
        df_pump = df.xs((site_id, pump_id), level=('site_id', 'pump_id')).copy()

        # Check if necessary columns exist for this pump
        required_columns = [target_column, annotation_column]
        missing_columns = [col for col in required_columns if col not in df_pump.columns]
        if missing_columns:
            print(f"Columns {missing_columns} not found for site_id {site_id}, pump_id {pump_id}. Skipping.")
            continue

        # Prepare the output DataFrame
        df_output = df_pump[[target_column, annotation_column]].copy()
        # df_output.rename(columns={annotation_column: 'downtime'}, inplace=True)
        # df_output['downtime'] = df_output['downtime'].astype(int)

        # Reset index to include 'Timestamp' as a column
        df_output.reset_index(inplace=True)

        # **Ensure that 'Timestamp' is a datetime with timezone awareness**
        if df_output['timestamp'].dtype == 'datetime64[ns]':
            df_output['timestamp'] = df_output['timestamp'].dt.tz_localize('UTC')

        # **Check if there is an active alert (an alert in the last 24 hours of data)**
        # Use the maximum timestamp in the data
        max_timestamp = df_output['timestamp'].max()
        last_24_hours = max_timestamp - pd.Timedelta(hours=24)

        # Now, check if there are any anomalies in the last 24 hours of data
        recent_anomalies = df_output[(df_output['timestamp'] >= last_24_hours) & (df_output['anomaly'] == 1)]
        if not recent_anomalies.empty:
            downtime_suffix = '_anomaly'
            downtime_path_suffix = 'anomaly'
        else:
            downtime_suffix = ''
            downtime_path_suffix = 'nonanomaly'

        # Construct the file name
        filename = f"site_{site_name_clean}_{site_id}_pump_{int(pump_id)}_BEPPercent_{date}{downtime_suffix}.csv"

        # Full path to save the CSV file
        site_output_path = os.path.join(output_base_path, date, downtime_path_suffix)
        os.makedirs(site_output_path, exist_ok=True)
        filepath = os.path.join(site_output_path, filename)

        # Output column order
        desired_column_order = ['timestamp', target_column,'anomaly']
        # Save to CSV
        df_output.to_csv(filepath, index=False, columns=desired_column_order)

        print(f"Saved data for site {site_id}, pump {pump_id} to {filepath}")